# Demonstration of a simple evaluation pipeline

Polara supports various evaluation regimes and can be tuned flexibly to achieve the setup you need.
<div class="alert alert-block alert-warning"> Note, that particular evaluation settings may not be directly supported by some models and may require a certain model modification.</div>

For example, matrix factorization models are not directly applicable in the *warm-start scenario* (when test users are not present in the training set) until the *folding-in* technique is implemented for generating recommendations. Keep this in mind when creating your custom solutions.  

<div class="alert alert-block alert-info">In this example we demonstrate basic evaluation of two well-known factorization models - `PureSVD` and `iALS` - in both standard and warm-start settings.</div>

The models can be easily tested in both scenarios without any modification due to automatic handling of the warm-start case, provided by Polara.

## Preparing Movielens-1M data

In [1]:
from polara.recommender.data import RecommenderData
from polara.datasets.movielens import get_movielens_data

In [2]:
data = get_movielens_data() # will automatically download it, or you can specify a path to the local copy
data.head()

,userid,movieid,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [3]:
data.shape

(1000209, 3)

In [4]:
data_model = RecommenderData(data, 'userid', 'movieid', 'rating', seed=0)
data_model.get_configuration()

{'holdout_size': 3,
 'negative_prediction': False,
 'permute_tops': False,
 'random_holdout': False,
 'shuffle_data': False,
 'test_fold': 5,
 'test_ratio': 0.2,
 'test_sample': None,
 'warm_start': True}

## Standard scenario with known users

In [5]:
data_model.random_holdout = True # allow not only top-rated items in evaluation, this reduces evaluation biases
data_model.warm_start = False # standard case
data_model.prepare()

Preparing data...
Done.


Let's check for demonstration purposes that all test users are present in the training set:

In [6]:
data_model.test.holdout['userid'].isin(data_model.index.userid.training.new).all()

True

### PureSVD

In [7]:
from polara.recommender.models import SVDModel

In [8]:
svd = SVDModel(data_model) # create model
svd.switch_positive = 4 # mark ratings below 4 as negative feedback and treat them accordingly in evaluation
svd.build() # fit model
svd.evaluate() # by default it calculates the total number of hits

PureSVD training time: 0.1370795875410451s


Hits(true_positive=512, false_positive=112, true_negative=1164, false_negative=1836)

### iALS

In [9]:
# implicit library must be installed separately, follow instructions at https://github.com/benfred/implicit 
from polara.recommender.external.implicit.ialswrapper import ImplicitALS

In [10]:
als = ImplicitALS(data_model) # create model
als.switch_positive = 4 # same as for PureSVD, affects only evaluation
als.build()
als.evaluate()

iALS training time: 1.5761851485s


Hits(true_positive=514, false_positive=116, true_negative=1160, false_negative=1834)

The maximum possible number of correct recomendations is:

In [11]:
data_model.test.holdout.query('rating>=4').shape[0]

2348

Both models correctly retrieve around a quarter of all items. Let's look on the averaged relevance scores:

In [12]:
svd.evaluate('relevance')

Relevance(precision=0.34864790286975716, recall=0.20088300220750549, fallout=0.056015452538631341, specifity=0.62279249448123619, miss_rate=0.72875275938189843)

In [13]:
als.evaluate('relevance')

Relevance(precision=0.34892384105960267, recall=0.20212472406181015, fallout=0.058912803532008826, specifity=0.61989514348785868, miss_rate=0.72751103752759372)

## Warm-start scenario

This will split test users from the training data.

In [14]:
data_model.warm_start = True # warm-start case
data_model.prepare()

Preparing data...
19 unique movieid's within 26 testset interactions were filtered. Reason: not in the training data.
1 unique movieid's within 1 holdout interactions were filtered. Reason: not in the training data.
1 of 1208 userid's were filtered out from holdout. Reason: not enough items.
1 userid's were filtered out from testset. Reason: inconsistent with holdout.


There's no intersection between test and training users:

In [15]:
data_model.index.userid.test.old.isin(data_model.index.userid.training.old).any()

False

<div class="alert alert-block alert-info">Polara makes a certain level of efforts to preserve data sanity and consistency.</div>

For example, as can be seen from the log message above, it filters out the items that are happen to be in the test split but are not a part of the training set.

### PureSVD

In [16]:
svd.build()
svd.evaluate()

PureSVD training time: 0.09909787366876799s


Hits(true_positive=515, false_positive=111, true_negative=1164, false_negative=1831)

Note, that you do not have to recreate the models as they operate on top of the `data_model` instance.  
In fact, the state change in `data_model` is synchronized with the dependent models' states. It will force models to rebuild themselves even if you do not explicitly specify it (even though it is recommended to be explicit to conform with the Zen of Python).

### iALS

In [17]:
als.evaluate()

iALS model is not ready. Rebuilding.
iALS training time: 1.25310956069s


Hits(true_positive=509, false_positive=117, true_negative=1158, false_negative=1837)

The maximum possible number of correct recomendations is:

In [18]:
data_model.test.holdout.query('rating>=4').shape[0]

2346

Check relevance scores:

In [19]:
svd.evaluate('relevance')

Relevance(precision=0.34907484120408727, recall=0.2020160176746755, fallout=0.056614194973764152, specifity=0.62192764429715541, miss_rate=0.7275614471140569)

In [20]:
als.evaluate('relevance')

Relevance(precision=0.34727975697321178, recall=0.19884009942004971, fallout=0.059375863021264838, specifity=0.61916597624965475, miss_rate=0.73073736536868272)

## Final remark

In these experiments we used the default settings for both models and, therefore, the results may not be neccessarily optimal. 
Also note, that the output of SVD is deterministic, while iALS tends to provide varying results, spread around some average value.  
<div class="alert alert-block alert-warning">In order to provide a fair comparison of these models one have to run a full cross-validation experiment with parameters tuning and confidence interval estimation.</div>

It can be easily done within Polara as well and will be covered in a separate guide.